In [125]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [126]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [127]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66, 41,
        58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20, 58,
        41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42, 55,
        55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60, 55,
         1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,  1,
        43, 55, 53, 58, 41, 44, 45,  0, 27, 65])


In [128]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[58, 65,  6,  1, 49, 54,  1, 59],
        [54, 47,  1, 42, 65,  1, 41,  1],
        [52, 49, 60, 60, 52, 45,  1, 41],
        [53, 41, 65,  1, 43, 55, 53, 45]])
targets:
tensor([[65,  6,  1, 49, 54,  1, 59, 55],
        [47,  1, 42, 65,  1, 41,  1, 63],
        [49, 60, 60, 52, 45,  1, 41, 54],
        [41, 65,  1, 43, 55, 53, 45,  1]])


In [129]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [130]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generator(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim= -1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generator(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


,KdVERE;s:Z0Lsu;Sz E9U&;JRE9.htpdko eVo,Lobz.My”gMN?)-)Pb)D’s-MO
WEvD0yaM,DuniEAR(‘eD)‘TIQvDYYHygz)z!WPDQ-,L&QBw.—j:qdgMg’ Q?d‘gbs“xU:?EpZhz);-,X?ZPrM,JPH0Pb?ZUC0”rQBxqTXcbxLmZ0IM!jEGa)MIKpGKt9suFxmB?)vhhi jEs’;-CB!(9s.ri:Eo;;S.1x“pqd’Dn;mlSEODYmPGZeWdo wjXZ‘UmM1yvD0K?’ s.Jq:c—S)z”.1ylE&gbsuTF;N;G&T-0Sh:pejvLX((FbNGdkFbuniqxk,yLZu NyLoX9R-“G.c’vPp!xTDd0‘kWtpqDZv!&Cafgja?ZIH&Z.C—(K?:UvVeRAvh.z!’0PgFdqdVViLToJP-“oL9.u BxpgCC!1
)d’.—VEb1V:M,DFGQ-0;G&“Qe?:ejRFmR“!’H‘z9(1OST!zPsgmb— EpcNF,UHQz.I1‘qe:


In [133]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.823143482208252, val loss: 4.838193416595459
step: 250, train loss: 4.776322364807129, val loss: 4.758362293243408
step: 500, train loss: 4.7193803787231445, val loss: 4.706659317016602
step: 750, train loss: 4.63790225982666, val loss: 4.641159534454346
4.690311431884766


In [134]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generator(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


-
vER(UC—X“Rl’.Ry)WE0OJfzQ?OwFl!&a’T1’?hOc-“Q&1E:?Zb‘FOh9shRUV0—cy EvKU!c!‘eEKWIE:t?RPiZ”d’!BbNjgCair);Y0OaiJF‘LTo;mRHz0xN’ lf’ Nqni’
b)O,
C—
ndkAEpn&)AvQ((&—dwss“01VLR“QbgKwHR?” 1koXz!CxrtpL’C—SpL!’’ h‘so “kj0“:B
 dVrjqbNJEgzQ’w0UwkjN!c FbuK?I—!lt&JM-C—T-rJJ:tldCrQ-CjOsdhtbzcU”ri9zRablM jfdxq” oniRqzVi&Qd1DYX)PH&Yy—gZPGH9k—F,.
Fq!z!:iwFKGEW—tpRFmR“ZHhLdVoks‘g?”gmRDaXpleL9xmdV9h‘WmsinKfyqkv”M-l?;O&vw(;o;:1DZEamdkfU!nunhOZZf?C(‘&TTdqd0EL—
blU”rIfFCQBWVFbWgaV?ZX“pnU!zn’f!“H9Fm s!Ul!(Xf&YBXvWEpgCwg
